
# Reduction

The goal of this exercise is to teach you about shared memory and parallel programming on a thread block level in the CUDA programming model.

Please follow these steps:

**Step 1**. Read through the entire notebook and run all cells at least once

**Step 2**. Implement the missing part of the kernel code, following the instructions in the comments. The goal is that shared memory is used to sum the per-thread partial sums into a single per-thread-block partial sum

Hints:
* The kernel uses a so-called grid strided loop, where all threads in the grid cooperatively iterate over the problem domain. Therefore, the number of thread blocks does not depend on the size of the to be summed array. All threads from all blocks first iterate (collectively) over the problem size (n) to obtain a per-thread partial sum.
* Within the thread block the per-thread partial sums are to be combined into a single per-thread-block partial sum.
* Each thread block stores its partial sum to out_array[blockIdx.x]
* The kernel is called twice, the second kernel is executed with only one thread block to combine all per-block partial sums to a single, global sum for the whole input array



In [ ]:
import numpy as np
import cupy as cp

Executing the following cell block writes its contents to the file "reduction.cu". Please read the comments to see what you need you to do to complete the implementation of this kernel.

In [ ]:
%%writefile reduction.cu

#define block_size_x 256

__global__ void reduce_kernel(float *out_array, float *in_array, int n) {

    int ti = threadIdx.x;
    int x = blockIdx.x * block_size_x + threadIdx.x;
    int step_size = gridDim.x * block_size_x;
    float sum = 0.0f;

    //cooperatively (with all threads in all thread blocks) iterate over input array
    for (int i=x; i<n; i+=step_size) {
        sum += in_array[i];
    }

    //at this point we have reduced the number of values to be summed from n to
    //the total number of threads in all thread blocks combined

    //the goal is now to reduce the values within each thread block to a single
    //value per thread block for this we will need shared memory

    //declare shared memory array, how much shared memory do we need?
    //__shared__ float ...;

    //make every thread store its thread-local sum to the array in shared memory
    //... = sum;
    
    //now let's call syncthreads() to make sure all threads have finished
    //storing their local sums to shared memory
    __syncthreads();

    //now this interesting looking loop will do the following:
    //it iterates over the block_size_x with the following values for s:
    //if block_size_x is 256, 's' will be powers of 2 from 128, 64, 32, down to 1.
    //these decreasing offsets can be used to reduce the number
    //of values within the thread block in only a few steps.
    #pragma unroll
    for (unsigned int s=block_size_x/2; s>0; s/=2) {

        //you are to write the code inside this loop such that
        //threads will add the sums of other threads that are 's' away
        //do this iteratively such that together the threads compute the
        //sum of all thread-local sums 

        //use shared memory to access the values of other threads
        //and store the new value in shared memory to be used in the next round
        //be careful that values that should be read are
        //not overwritten before they are read
        //make sure to call __syncthreads() when needed
    }

    //write back one value per thread block
    if (ti == 0) {
        //out_array[blockIdx.x] = ;  //store the per-thread block reduced value to global memory
    }
}

The following prepares the input and output data of our kernel.

In [ ]:
# Setup kernel launch parameters
block_size_x = 256
num_blocks = 2048
grid_size = (num_blocks, 1, 1)
block_size = (block_size_x, 1, 1)

# Create input and output data
size = np.int32(5e7)
input_cpu = np.random.randn(size).astype(np.float32) + 0.00000001

# Allocate GPU memory
input_gpu = cp.asarray(input_cpu)
out_gpu = cp.zeros(num_blocks, dtype=cp.float32)

We compute a reference answer in Python to check if our GPU kernel returns the correct result.

In [ ]:
%%time

#compute reference sum in Python
npsum = np.sum(input_cpu)

The following cell compiles and runs the kernel and checks if the output is correct. This is all in one cell so that you only need to rerun this cell after modifying the kernel code in the cell above.

In [ ]:
# Read kernel into string
with open('reduction.cu', 'r') as f:
    kernel_string = f.read()

# Compile CUDA kernel
reduction_gpu = cp.RawKernel(kernel_string, "reduce_kernel")

# Clear the GPU output array for correctness checks
out_gpu = cp.zeros(num_blocks, dtype=cupy.float32)

# Execute athe first partial reduction
reduction_gpu(grid_size, block_size, (out_gpu, input_gpu, size))
# Execute the second and final reduction
reduction_gpu((1, 1, 1), block_size, (out_gpu, out_gpu, 2048))

print("PASSED" if np.absolute(npsum - gpu_sum) < 10.0 else "FAILED")